In [1]:
from PIL import Image
import requests
import torch
from torchvision import io
from typing import Dict
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor

In [2]:
# Load the model in half-precision on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
# _preprocess in Qwen2VLImageProcessor
# image can be visualized till `patches`
# data_format == ChannelDimension.LAST -> False
# patches.shape[0] % self.temporal_patch_size != 0 -> True
# TODO: why does processed_image look different with imshow?

# patches -> 2, 3, 1372, 2044 (image repeated across dim=0, c, h, w)
# grid_t = 1, grid_h = 98, grid_w = 146
# temporal_patch_size = 2
# patch_size = 14
# merge_size = 2
# resized_height = 1372
# resized_width = 2044
# channel = 3

#processor

In [ ]:
# llava image inputs -> channel x height x wdith
#  

In [1]:
import numpy as np

In [14]:
height = 1372
width = 2044
channel = 3
temporal_patch_size = 2
patch_size = 14
merge_size = 2
patches = np.random.rand(2, channel, height, width)

grid_t = patches.shape[0] // temporal_patch_size
grid_h = height // patch_size
grid_w = width // patch_size

patches = patches.reshape(
    grid_t,
    temporal_patch_size,
    channel,
    grid_h // merge_size,
    merge_size,
    patch_size,
    grid_w // merge_size,
    merge_size,
    patch_size,
)
print(patches.shape)
patches = patches.transpose(0, 3, 6, 4, 7, 2, 1, 5, 8)
print(patches.shape)
flatten_patches = patches.reshape(
    grid_t * grid_h * grid_w, channel * temporal_patch_size * patch_size * patch_size
)
print(flatten_patches.shape)

(1, 2, 3, 49, 2, 14, 73, 2, 14)
(1, 49, 73, 2, 2, 3, 2, 14, 14)
(14308, 1176)


In [ ]:
# TODO: how is flatten_patches used in the model
# TODO: map flatten_patches to -> 2, channel, height, width

In [4]:
# Image
url = "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
image = Image.open(requests.get(url, stream=True).raw)

conversation = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]


# Preprocess the inputs
text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
# Excepted output: '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>Describe this image.<|im_end|>\n<|im_start|>assistant\n'

inputs = processor(
    text=[text_prompt], images=[image], padding=True, return_tensors="pt"
)
inputs = inputs.to(model.device)

14 2 1372 2044 3


NameError: name 'quit' is not defined

In [8]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'])

In [5]:
# Inference: Generation of the output
output_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]
output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)
print(output_text)

["The image depicts a serene beach scene with a woman and her dog enjoying a moment together. The woman is sitting on the sandy beach, facing the ocean, and appears to be engaging in a playful activity with her dog. The dog, which is a large breed, is sitting on its hind legs and extending its front paw towards the woman's hand. The woman is smiling and seems to be enjoying the interaction, as she is holding out her hand for the dog to touch. \n\nThe woman is dressed in casual attire, wearing a plaid shirt and dark pants. She has long hair that is flowing freely in the breeze. The dog is"]
